In [ ]:
import pandas as pd
from nltk import word_tokenize, pos_tag, ne_chunk
from unidecode import unidecode
import nltk

In [ ]:
tweets = pd.read_csv('/mnt/idms/fberes/network/roland_garros/data/rg17_tweets_eng.csv',sep='|')
tweets['text']=tweets['text'].apply(lambda x: unidecode(unicode(x, encoding = "utf-8")))
tweets[:10]

In [ ]:
def get_labeled_words(text,label):
    wordlist=[]
    for subtree in text.subtrees(filter=lambda t: t.label()==label):
        for leave in subtree.leaves():
            wordlist.append(leave[0])
    return wordlist

In [ ]:
def label_table(data,label):
    word_list = [x for sublist in data for x in sublist]
    if word_list:
        word_count = {word:word_list.count(word) for word in set(word_list)}
        df=pd.DataFrame.from_dict(word_count, orient='index')
        df.columns=['count']
        df['label']=label
        df.index.name='word'
    else:
        df=pd.DataFrame()
    return df

In [ ]:
num = 1000
tweets['groupby_col'] = [x/1000 for x in range(len(tweets))]
grouped_tweets = tweets.groupby('groupby_col')[['text']].aggregate(lambda x: ''.join(x))

In [ ]:
%%time
grouped_tweets['ne_chunk']=grouped_tweets['text'].apply(lambda x: ne_chunk(pos_tag(word_tokenize(x))))
labels=['FACILITY', 'GPE', 'GSP', 'LOCATION', 'ORGANIZATION', 'PERSON']
labeled_words=pd.DataFrame()
for label in labels:
    labeled_word_list=grouped_tweets['ne_chunk'].apply(lambda x: get_labeled_words(x,label))
    labeled_df = label_table(labeled_word_list,label)
    labeled_words = labeled_words.append(labeled_df)

In [ ]:
labeled_words

In [ ]:
labeled_words.to_csv('/mnt/idms/fberes/network/combined_occ/nltk_labeled_words.csv',sep='|',index=True,header=True)